In [8]:
import time
import requests
import pandas as pd
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
from IPython.display import clear_output
import requests
import pandas_ta as ta
from rdp import rdp
import talib
import numpy as np
import pandas_ta as ta
from scipy.stats import zscore
from scipy.signal import savgol_filter
from datetime import datetime, date, time as dt_time, timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# ── CONFIG ─────────────────────────────────────────────────────────────────
API_KEY = "vBRy5un9PuHfxFj1IrHpfg8a2RS57jE9"

# https://github.com/polygon-io/client-python/blob/master/examples/websocket/stocks-ws.py

In [9]:
def fetch_polygon_data(ticker: str, api_key: str,
                              start_date: date,
                              end_date: date,
                              start_time: dt_time,
                              end_time: dt_time = dt_time(16, 0),
                              multiplier: int = 1,
                              timespan: str = "minute",
                              limit: int = 50000) -> pd.DataFrame:
    """
    Fetches intraday minute-level OHLCV data for a given ticker from Polygon.io.

    Args:
        ticker (str): Stock ticker symbol (e.g., "TSLA").
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch data.
        end_date (date): The date for which to end fetch data.
        start_time (datetime.time): Start of time window in EST.
        end_time (datetime.time): End of time window in EST.
        multiplier (int): Interval multiplier (default 1 minute).
        timespan (str): Time unit ("minute", "hour", etc.).
        limit (int): Maximum number of results to retrieve.

    Returns:
        pd.DataFrame: A filtered DataFrame with renamed columns and timestamps in EST.
    """
    ET_ZONE = ZoneInfo("America/New_York")

    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{start_date}/{end_date}"
    params = {
        "apiKey": api_key,
        "adjusted": "true",
        "sort": "asc",
        "limit": limit
    }

    session = requests.Session()
    response = session.get(url, params=params)
    response.raise_for_status()

    bars = response.json().get("results", [])
    df = pd.DataFrame(bars)

    if df.empty:
        return pd.DataFrame()  # Return empty DataFrame if no data

    df["TIME_UTC"] = pd.to_datetime(df["t"], unit="ms", utc=True)
    df["TIME_EST"] = df["TIME_UTC"].dt.tz_convert(ET_ZONE)
    del df["TIME_UTC"]

    # Define time window
    start_dt = datetime.combine(start_date, start_time, tzinfo=ET_ZONE)
    end_dt = datetime.combine(end_date, end_time, tzinfo=ET_ZONE)

    # Filter and rename
    df = df[(df["TIME_EST"] >= start_dt) & (df["TIME_EST"] <= end_dt)].copy()
    df.rename(columns={
        "o": "open",
        "h": "high",
        "l": "low",
        "c": "close",
        "v": "volume",
        "n": "trades",
        "vw": "vwap"
    }, inplace=True)

    return df


In [10]:


def add_indicators_and_signals(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds technical indicators, buy conditions, and entry signal column to the given DataFrame.
    Assumes columns: 'open', 'high', 'low', 'close', 'volume'


    Returns:
        DataFrame with new columns added.
    """

    # Bollinger Bands_________________________________________________________
    # ________________________________________________________________________

    up, _, dn = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=2, nbdevdn=2)
    df['BB_up_2'], df['BB_dn_2'] = up, dn

    up, _, dn = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=1, nbdevdn=1)
    df['BB_up_1'], df['BB_dn_1'] = up, dn

    # VWAP (Cumulative approximation)_________________________________________
    # ________________________________________________________________________

    start_time = dt_time(4, 0)   # ex.,4:00 AM ET
    end_time = dt_time(23, 59)  

    mask = (df['TIME_EST'].dt.time >= start_time) & (df['TIME_EST'].dt.time <= end_time)
    df_trading = df.loc[mask].copy()

    df.loc[mask, 'VWAP_Cum'] = ((df.loc[mask, 'close'] * df.loc[mask, 'volume']).groupby(df.loc[mask, 'TIME_EST'].dt.date).cumsum()
                                /
                                df.loc[mask, 'volume'].groupby(df.loc[mask, 'TIME_EST'].dt.date).cumsum())

    # EMA, MACD, ATR, SMA ____________________________________________________
    # ________________________________________________________________________

    macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'], df['Signal'], df['MACD_hist'] = macd, sig, hist

    # EMA, MACD, ATR, SMA     ----------------------------
    df['EMA9'] = talib.EMA(df.close.values, timeperiod=9)
    df['EMA20'] = talib.EMA(df.close.values, timeperiod=20)
    df['EMA50'] = talib.EMA(df.close.values, timeperiod=50)
    df['EMA200'] = talib.EMA(df.close.values, timeperiod=200)

    macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'], df['Signal'], df['MACD_hist'] = macd, sig, hist
    df["MACD_hist_smooth_EMA"] = df["MACD_hist"].ewm(span=5, adjust=False).mean()

    df['SMA5'] = talib.SMA(df.close.values, timeperiod=5)
    df['SMA20'] = talib.SMA(df.close.values, timeperiod=20)
    df['SMA50'] = talib.SMA(df.close.values, timeperiod=50)
    df['SMA200'] = talib.SMA(df.close.values, timeperiod=200)

    # RSI                     ----------------------------  
    df['RSI'] = talib.RSI(df.close.values, timeperiod=14)
    
    # TSI                     ________________________________________________
    # ________________________________________________________________________

    tsi_df = ta.tsi(df['close'], long=25*4, short=13*4, signal=8*4)
    tsi_df.index = df.index  # Align index to match df
    tsi_df.columns = ['TSI', 'TSI_signal']
    df = df.join(tsi_df)
    
    # Confriming Signal_______________________________________________________
    # ________________________________________________________________________
    
    df['SMA200_VL']=df['SMA200'].diff()
    df['SMA50_VL']=df['SMA50'].diff()

    # zscored_signal = zscore(df[['SMA50_VL','SMA200_VL']].values, axis=0, ddof=0, nan_policy='omit')
    # df[['SMA50_VL_ZSCR', 'SMA200_VL_ZSCR']] = zscored_signal
    
    # --- Daily z-score of the diff columns (resets each day) ---
    _seg = df.loc[mask] 
    
    out = (
        _seg
        .groupby(_seg['TIME_EST'].dt.date, group_keys=False)[['SMA50_VL','SMA200_VL']]
        .apply(lambda g: pd.DataFrame(
            zscore(g.to_numpy(), axis=0, ddof=0, nan_policy='omit'),
            index=g.index,
            columns=['SMA50_VL_ZSCR_DAY','SMA200_VL_ZSCR_DAY']
        ))
    )

    # Assign back aligned by index
    df[['SMA50_VL_ZSCR_DAY','SMA200_VL_ZSCR_DAY']] = np.nan
    df.loc[out.index, ['SMA50_VL_ZSCR_DAY','SMA200_VL_ZSCR_DAY']] = out
    
    # smoothed Signal         ----------------------------
    df['SMA50_VL_smooth'] = savgol_filter(df['SMA50_VL'], window_length=150, polyorder=7)
    df['SMA200_VL_smooth'] = savgol_filter(df['SMA200_VL'], window_length=150, polyorder=7)

    zscored_signal= zscore(df[['SMA50_VL_smooth','SMA200_VL_smooth']].values, axis=0, ddof=0, nan_policy='omit')
    df[['SMA50_VL_ZSCR_smooth', 'SMA200_VL_ZSCR_smooth']] = zscored_signal 

    # --- Daily z-score of the smoothed columns (resets each day) ---
    _seg = df.loc[mask] 
    
    out = (
        _seg
        .groupby(_seg['TIME_EST'].dt.date, group_keys=False)[['SMA50_VL_smooth','SMA200_VL_smooth']]
        .apply(lambda g: pd.DataFrame(
            zscore(g.to_numpy(), axis=0, ddof=0, nan_policy='omit'),
            index=g.index,
            columns=['SMA50_VL_ZSCR_smooth_DAY','SMA200_VL_ZSCR_smooth_DAY']
        ))
    )
    
    # Assign back aligned by index
    df[['SMA50_VL_ZSCR_smooth_DAY','SMA200_VL_ZSCR_smooth_DAY']] = np.nan
    df.loc[out.index, ['SMA50_VL_ZSCR_smooth_DAY','SMA200_VL_ZSCR_smooth_DAY']] = out

    return df
    

In [11]:
from datetime import date
import time
from typing import List, Dict

def fetch_multiple_tickers_data(ticker: List[str],
                                api_key: str,
                                start_date: date,
                                end_date: date,
                                start_time: dt_time,
                                end_time: dt_time,
                                multiplier: int,
                                timespan: str,
                                limit: int ,
                                delay: float = 0.25) -> Dict[str, pd.DataFrame]:
    """
    Fetch minute-level Polygon data for multiple tickers on a given date.

    Args:
        ticker (List[str]): List of stock ticker symbols.
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch intraday data.
        delay (float): Seconds to wait between API calls (rate limit buffer).

    Returns:
        Dict[str, pd.DataFrame]: Dictionary of ticker symbol → DataFrame.
    """
    all_data = {}

    for ticker in ticker:
        try:
            df = fetch_polygon_data(
                ticker=ticker,
                api_key=API_KEY,
                start_date=start_date,
                end_date=end_date,
                start_time=start_time,
                end_time=end_time,
                multiplier=multiplier,
                timespan=timespan,
                limit=limit
                )
            if not df.empty:
                
                df = add_indicators_and_signals(df)

                all_data[ticker] = df

            print(f"✅ {ticker} - fetched {len(df)} rows")

        except Exception as e:
            print(f"❌ Error fetching {ticker}: {e}")

        time.sleep(delay) # ⏱️ Avoid hitting rate limits

    return all_data


In [12]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_intraday(df, ticker):
    fig = make_subplots(
        rows=6, cols=1,
        shared_xaxes=True,
        specs=[
            [{"secondary_y": True}],  # Row 1
            [{}], [{}], [{}], [{}], [{}]  # Rows 2-6
        ],
        row_heights=[0.3, 0.15, 0.15, 0.08, 0.15, 0.15],
        vertical_spacing=0.02,
        subplot_titles=("Price + BB + VWAP", "RSI5", "MACD Hist, MACD & Signal", "TSI", "V", "A")
    )

    # Row 1: Price + BB + VWAP
    fig.add_trace(go.Candlestick(
        x=df["TIME_EST"], open=df["open"], high=df["high"],
        low=df["low"], close=df["close"], name="Price"
    ), row=1, col=1, secondary_y=False)
    
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["BB_up_1"], line=dict(dash="dash"), name="BB Up 1"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["BB_dn_1"], fill="tonexty",fillcolor="rgba(200,200,200,0.2)", name="BB Dn 1"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["VWAP_Cum"], line=dict(dash="dot"), name="VWAP"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["EMA200"], line=dict(color="orange"), name="EMA200"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["SMA200"], line=dict(color="blue"), name="SMA200"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["SMA50"], line=dict(color="black"), name="SMA50"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["EMA50"], line=dict(color="khaki"), name="EMA50"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["SMA5"], line=dict(color="gray"), name="SMA5"), row=1, col=1)

    # Row 1: Volume zones
    df['volume_color'] = np.where(df['close'] > df['close'].shift(1), 'green',
                                  np.where(df['close'] < df['close'].shift(1), 'red', 'gray'))
    fig.add_trace(go.Bar(x=df["TIME_EST"], y=df["volume"], name="Vol",
                         marker_color=df["volume_color"], width=1000, opacity=.3, marker_line_width=0),
                  row=1, col=1, secondary_y=True)

    # Row 2: RSI5
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["RSI"], name="RSI", line=dict(color="olive")), row=2, col=1)
    for level in [70, 50, 30]:
        fig.add_hline(y=level, line_dash="dash", row=2, col=1)

    # Row 3: MACD Histogram, MACD & Signal
    hist_colors = ['green' if val >= 0 else 'red' for val in df["MACD_hist"]]
    fig.add_trace(go.Bar(x=df["TIME_EST"], y=df["MACD_hist"],
                         marker=dict(color=hist_colors, line=dict(color=hist_colors, width=1)),
                         name='MACD Hist'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['MACD'], line=dict(color='olive', width=2), name='MACD'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["Signal"], line=dict(color="salmon", width=1), name="Signal"), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["MACD_hist_smooth_EMA"],
                             line=dict(color="brown", width=1), name="MACD_hist_smooth_EMA"), row=3, col=1)

    # Row 4: TSI
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['TSI'], line=dict(color='red', width=2), name='TSI'), row=4, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['TSI_signal'], line=dict(color='blue', width=2), name='TSI_signal'), row=4, col=1)
    for level in [-25, 0, 25]:
        fig.add_hline(y=level, line_dash="dash", row=4, col=1)

    # Row 5: V-A
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA50_VL_ZSCR_DAY'], line=dict(color='black', width=1), name='SMA50_VL_ZSCR_DAY'), row=5, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA200_VL_ZSCR_DAY'], line=dict(color='blue', width=1), name='SMA200_VL_ZSCR_DAY'), row=5, col=1)
    fig.add_hline(y=0, line_dash="dash", row=5, col=1)


    # Row 6: V-A AC
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA50_VL_ZSCR_smooth'], line=dict(color='rgba(0, 0, 0, 0.15)', width=1), name='SMA50_VL_ZSCR_smooth'), row=6, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA200_VL_ZSCR_smooth'], line=dict(color='rgba(70, 130, 180, 0.35)', width=1), name='SMA200_VL_ZSCR_smooth'), row=6, col=1)
    fig.add_hline(y=0, line_dash="dash", row=6, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA50_VL_ZSCR_smooth_DAY'], line=dict(color='black', width=1), name='SMA50_VL_ZSCR_smooth_DAY'), row=6, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA200_VL_ZSCR_smooth_DAY'], line=dict(color='blue', width=1), name='SMA200_VL_ZSCR_smooth_DAY'), row=6, col=1)

    # Layout
    fig.update_layout(
        title=f"Intraday ET for {ticker} on {df['TIME_EST'].dt.date.iloc[-1]}",
        xaxis_rangeslider_visible=False,
        height=900, width=1000,
        paper_bgcolor="white", plot_bgcolor="white",showlegend=False  
    )

    # Gridlines
    fig.update_xaxes(showgrid=True, gridcolor='lightgray', rangeslider_visible=False)
    fig.update_yaxes(showgrid=True, gridcolor='lightgray')

    # Add vertical dashed green lines at buy conditions
    if "buy_condition" in df.columns:
        for t in df.loc[df["buy_condition"], "TIME_EST"]:
            fig.add_vline(x=t, line=dict(color='green', width=1, dash='dash'))

    fig.update_layout(
        title = f"{ticker} on {df['TIME_EST'].iloc[-1].strftime('%Y-%m-%d %H:%M:%S %Z')} | Last Close: {df['close'].iloc[-1]:.2f}",

        xaxis_rangeslider_visible=False,height=900, width=1000,paper_bgcolor="white", plot_bgcolor="white")

    return fig 

    # fig.show()


In [13]:
# Add example 100 tickers

# ===== Inputs =====
# Add example 100 tickers

TICKERS=['PLTR']

# Time window for filtering intraday bars (Eastern Time)

# ===== Inputs =====
start_date_DATA = date(2025, 11, 18)
end_date_DATA = date(2025, 11, 18)

start_time_DATA = dt_time(0, 0)   # ex.,4:00 AM ET
end_time_DATA = dt_time(20, 0)    #ex.,4:00 PM ET
#--------------------------------------------------
start_date_TRADE = date(2025, 11, 18)
end_date_TRADE = date(2025, 11, 18)

start_time_TRADE = dt_time(0, 0)   # ex.,4:00 AM ET
end_time_TRADE = dt_time(23, 59)    #ex.,4:00 PM ET


<!-- # from IPython.display import clear_output
import time
import sys
import pandas as pd  # only if you print timestamps

REFRESH_EVERY = 55  # seconds

def run_analysis():
    # 🔽 Put your entire analysis/plot/table code here
    #===========================================================================================================================================

    
    # ─────────────────────────────────────────────────────────────────────
    # GET DATA
    # ─────────────────────────────────────────────────────────────────────
    multiplier = 1               #   e.g., 1-minute bars,  5-minute bars, etc.
    timespan = "minute"          #  "second", "minute", "hour", "day", etc.
    limit = 50000
    
    all_data_minute = fetch_multiple_tickers_data(ticker=TICKERS,api_key=API_KEY,start_date=start_date_DATA,end_date=end_date_DATA,start_time=start_time_DATA,end_time=end_time_DATA,multiplier=multiplier,timespan=timespan,limit=limit)
    
    # ─────────────────────────────────────────────────────────────────────
    # Do the plot
    # ─────────────────────────────────────────────────────────────────────
    for tic in TICKERS:
        # df_day = all_data_minute_processed.get(tic)
        df_day = all_data_minute.get(tic)
    
        if df_day is None or df_day.empty:
            print(f"⚠️ {tic}: no data"); 
            continue
    
        fig = plot_intraday(df=df_day, ticker=tic)
    
        # Show regardless of library
        if hasattr(fig, "show"):          # Plotly Figure or Matplotlib Figure has .show()
            fig.show()
        else:
            try:
                display(fig)              # fallback for objects with rich repr
            except Exception:
                pass
    
    

    
    #===========================================================================================================================================
    # (everything up to and including plt.show())
    pass

while True:
    # 1) Run and DISPLAY results
    run_analysis()

    # 2) Keep results visible with a countdown (no clearing yet)
    print("🔁 Auto-refresh at:", pd.Timestamp.now())
    for i in range(REFRESH_EVERY, 0, -1):
        sys.stdout.write(f"\rNext refresh in {i:2d}s ")
        sys.stdout.flush()
        time.sleep(1)

    # 3) NOW clear, so next loop starts fresh
    clear_output(wait=True)


 -->


In [ ]:
# from IPython.display import clear_output
import time
import sys
import pandas as pd  # only if you print timestamps

REFRESH_EVERY = 55  # seconds

def run_analysis():
    # 🔽 Put your entire analysis/plot/table code here
    #===========================================================================================================================================

    
    # ─────────────────────────────────────────────────────────────────────
    # GET DATA
    # ─────────────────────────────────────────────────────────────────────
    multiplier = 1               #   e.g., 1-minute bars,  5-minute bars, etc.
    timespan = "minute"          #  "second", "minute", "hour", "day", etc.
    limit = 50000
    
    all_data_minute = fetch_multiple_tickers_data(ticker=TICKERS,api_key=API_KEY,start_date=start_date_DATA,end_date=end_date_DATA,start_time=start_time_DATA,end_time=end_time_DATA,multiplier=multiplier,timespan=timespan,limit=limit)
    
    # ─────────────────────────────────────────────────────────────────────
    # Do the plot
    # ─────────────────────────────────────────────────────────────────────
    for tic in TICKERS:
        # df_day = all_data_minute_processed.get(tic)
        df_day = all_data_minute.get(tic)
    
        if df_day is None or df_day.empty:
            print(f"⚠️ {tic}: no data"); 
            continue
    
        fig = plot_intraday(df=df_day, ticker=tic)
    
        # Show regardless of library
        if hasattr(fig, "show"):          # Plotly Figure or Matplotlib Figure has .show()
            fig.show()
        else:
            try:
                display(fig)              # fallback for objects with rich repr
            except Exception:
                pass
    
    

    
    #===========================================================================================================================================
    # (everything up to and including plt.show())
    pass

while True:
    # 1) Run and DISPLAY results
    run_analysis()

    # 2) Keep results visible with a countdown (no clearing yet)
    print("🔁 Auto-refresh at:", pd.Timestamp.now())
    for i in range(REFRESH_EVERY, 0, -1):
        sys.stdout.write(f"\rNext refresh in {i:2d}s ")
        sys.stdout.flush()
        time.sleep(1)

    # 3) NOW clear, so next loop starts fresh
    clear_output(wait=True)





✅ PLTR - fetched 878 rows


🔁 Auto-refresh at: 2025-11-18 22:59:41.778086
Next refresh in 52s 